**Import Libraries**

In [1]:
import pandas as pd


from geopy.geocoders import Nominatim
#import geocoder

**Read Dataset**

In [2]:
#innovations = pd.read_parquet("geocoding_all_0424.parquet")
innovations = pd.read_parquet("sample_for_geocoding.parquet")

**Preprocessing (minimise time used for geocoding)**

In [3]:
#drop all duplicates
geocoded_innovations = innovations.drop_duplicates(subset = "user_location", keep ="first")
#drop all rows with lat/lon
geocoded_innovations = geocoded_innovations[geocoded_innovations["longitude"].isna() & geocoded_innovations["latitude"].isna()]
#only "user_location" column
geocoded_innovations = geocoded_innovations[["user_location"]]

**Geocoding**

In [4]:
geolocator = Nominatim(user_agent="Eric")

geocoded_innovations["geocoded"] = geocoded_innovations.user_location.apply(geolocator.geocode)

In [24]:
#Store lat and lon in separate columns 
geocoded_innovations['Lat'] = geocoded_innovations["geocoded"].apply(lambda x: x.latitude if x else None)
geocoded_innovations['Lon'] = geocoded_innovations["geocoded"].apply(lambda x: x.longitude if x else None)

**Afterprocessing**

In [27]:
#drop "user_location" column before joining two dataset (dropping column, which can be found in both datasets (apart from index))
geocoded_innovations.drop("user_location", axis=1, inplace = True)

#joining two dataframes (left join full dataset and geocoded subset)
result = innovations.join(geocoded_innovations)

#Populate NaN values in columns "geocoded", "Lat" and "Lon" of same locations
result["geocoded"] = (result["geocoded"].fillna(result.groupby("user_location")["geocoded"].transform("first")))
result["Lat"] = (result["Lat"].fillna(result.groupby("user_location")["Lat"].transform("first")))
result["Lon"] = (result["Lon"].fillna(result.groupby("user_location")["Lon"].transform("first")))

result



KeyError: "['user_location'] not found in axis"

In [26]:
result

,created_at,id,twitter_id,text,truncated,user_id,user_location,user_lang,longitude,latitude,...,inno_31,inno_32,inno_33,inno_34,inno_39,inno_44,innovation,geocoded,Lat,Lon
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
1901,2019-01-02 07:33:59+00:00,201901020033_0094,1080366580385660928,@Mowgli420 Resurge financiado por el PRI ( ant...,False,933447807020814336,1ra fila de defensa de la 4T!,es,NaN,NaN,...,False,False,False,False,False,True,1,None,NaN,NaN
13725,2019-01-02 11:20:14+00:00,201901020420_0027,1080423518054109184,RT @Alexxs34: @carlitosHALO1 @danigtrrz Bro so...,False,2517604120,Arganda del Rey,es,NaN,NaN,...,False,False,False,False,False,False,1,"(Arganda del Rey, Cuenca del Henares, Comunida...",40.300771,-3.438069
14249,2019-01-02 11:28:06+00:00,201901020428_0012,1080425497778155521,@JupaFonseca Guapérrimo☺️,False,2465476310,"Madrid, Comunidad de Madrid",es,NaN,NaN,...,False,False,False,False,False,True,1,"(Madrid, Área metropolitana de Madrid y Corred...",40.416705,-3.703582
14250,2019-01-02 11:28:06+00:00,201901020428_0013,1080425497778155521,@JupaFonseca Guapérrimo☺️,False,2465476310,"Madrid, Comunidad de Madrid",es,NaN,NaN,...,False,False,False,False,False,True,1,"(Madrid, Área metropolitana de Madrid y Corred...",40.416705,-3.703582
14468,2019-01-02 11:31:37+00:00,201901020431_0057,1080426382772056064,@carapuchinhagz Me pide me perdona,False,382825395,"Vigo, Galiza",es,NaN,NaN,...,False,False,True,False,False,False,1,"(Vigo, Pontevedra, Galicia, 36202, España, (42...",42.237660,-8.724721
26280,2019-01-02 13:42:31+00:00,201901020642_0093,1080459324818894848,RT @mparadalopez: Acompáñame a ver esta triste...,False,1042473141115674624,Ciudad Autónoma de Buenos Aire,en,NaN,NaN,...,False,False,False,False,False,False,1,"(Buenos Aires, Comuna 6, Ciudad Autónoma de Bu...",-34.607568,-58.437089
28498,2019-01-02 14:02:13+00:00,201901020702_0054,1080464282524049408,@AlbaBM12 bro sos famosa,False,1026116450879201280,14,es,NaN,NaN,...,False,False,False,False,False,False,1,"(Calvados, Normandie, France métropolitaine, F...",49.090765,-0.241395
29967,2019-01-02 14:14:30+00:00,201901020714_0104,1080467373709246465,Literalmente la palabra crush es moler/machaca...,True,2305925332,Ubicación,es,NaN,NaN,...,False,False,False,False,False,False,1,"(Ubicación apróximada Cerro Diablo, Inírida, G...",3.426562,-67.955360
31636,2019-01-02 14:27:17+00:00,201901020727_0075,1080470590719512576,RT @Panchicista: Cuando lo pides por AliExpres...,False,275944162,"Castelló, País Valencià",es,NaN,NaN,...,False,False,False,False,False,False,1,"(Castelló / Castellón, Comunitat Valenciana, E...",40.251857,-0.061505


**Interactive Visualisation**

In [7]:
import folium
from folium.plugins import HeatMapWithTime
import numpy as np
from numpy import nan
import math

In [30]:
#Removing all row, where Lat column has NaN values
nonan = result[result["Lat"].notna()]

In [33]:
lat_lon_list = []
for i in nonan["created_at"].unique():
    temp=[]
    for index, instance in nonan[nonan["created_at"] == i].iterrows():
        temp.append([instance["Lat"],instance["Lon"]])
    lat_lon_list.append(temp)


[[[40.3007708, -3.4380688]],
 [[40.4167047, -3.7035825], [40.4167047, -3.7035825]],
 [[42.2376602, -8.7247205]],
 [[-34.6075682, -58.4370894]],
 [[49.09076485, -0.24139505722798021]],
 [[3.4265617, -67.95536]],
 [[40.25185675, -0.061505110105748245]],
 [[51.1191931, -115.4244591]],
 [[-35.2340308, -63.5929581]],
 [[-34.604043, -58.3870845]],
 [[-7.1280388, 52.7448393]],
 [[-34.540349, -58.7157491]],
 [[39.2673015, -1.5492105]],
 [[-12.0522626, -77.1391133]],
 [[43.4192789, 39.9276453]],
 [[-33.0078585, -58.5103466]],
 [[-3.46800335, 31.899214070574452]],
 [[-34.787093049999996, -68.43818677312292]],
 [[40.3952888, -3.8643349]],
 [[40.9651572, -5.6640182]],
 [[-34.9206797, -57.9537638]],
 [[19.4326296, -99.1331785]],
 [[25.6950285, -100.4682393]],
 [[-33.0078585, -58.5103466]],
 [[-39.6812225, -68.9514866]],
 [[36.7213028, -4.4216366]],
 [[44.933143, 7.540121]],
 [[35.8978364, -106.3998742]],
 [[-31.4173391, -64.183319]],
 [[-24.97210025, -65.20386575329536]],
 [[-37.9977225, -57.548270

In [32]:
#converting it to datetime format
nonan["created_at"]= pd.to_datetime(nonan["created_at"])#creating a time index
time_index = []
for i in nonan["created_at"].unique():
    time_index.append(i)#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

<ipython-input-32-73367dad24e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonan["created_at"]= pd.to_datetime(nonan["created_at"])#creating a time index


In [34]:
m = folium.Map([47.32, 8.50], zoom_start=5, tiles = "CartoDB dark_matter", control_scale = True)

HeatMapWithTime(lat_lon_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=1).add_to(m)

m

In [ ]:
# Todo
# Geocode all dataframe
# add innovation selector to folium
# find better visualisation in folium. not heatmap but point
# Move coordinates for origninally geocoded tweets from columns "Latitude" and "Longitude" to "Lat" and "Lon"